In [ ]:
!pip install libtorrent --quiet

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
import os
import gc
import re
import time
import shutil
import libtorrent as lt

In [ ]:
# Set the desired timezone (e.g., "America/New_York")
desired_timezone = "Europe/Istanbul"

# Set the timezone using the environment variable
os.environ["TZ"] = desired_timezone

# Call tzset() to apply the changes
time.tzset()

In [ ]:
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
def validate_magnet_link(magnet_link, exact=False):
    """
    Validate a magnet link based on a specified pattern.

    Args:
    - magnet_link (str): The magnet link to be validated.
    - exact (bool): If True, match the entire string; otherwise, match partial.

    Returns:
    - bool: True if the magnet link is valid, False otherwise.
    """

    # Define the pattern based on exact match requirement
    pattern_str = (
        r"^magnet:\?xt=urn:[a-z0-9]+:[a-z0-9]{32,40}&dn=.+&tr=.+$"
        if exact
        else r"magnet:\?xt=urn:[a-z0-9]+:[a-z0-9]{32,40}&dn=.+&tr=.+"
    )

    # Compile the regular expression pattern
    pattern = re.compile(pattern_str, re.I | re.M)

    # Return True if the magnet link matches the pattern, otherwise False
    return bool(pattern.match(magnet_link))

In [ ]:
def load_magnet_links(file_name: str) -> set:
    """
    Load and validate magnet links from a file.

    Args:
    - file_name (str): Name of the file containing magnet links.

    Returns:
    - set: A set of valid magnet links.

    Raises:
    - FileNotFoundError: If the specified file is not found.
    """

    try:
        # Read and strip each line in the file
        with open(file_name) as f:
            lines = [line.strip() for line in f]

        # Filter out invalid magnet links
        magnet_links = [line for line in lines if validate_magnet_link(line)]

        # Display the number of valid magnet links loaded
        print(f"Loaded {len(magnet_links)} magnet links from {file_name}")

        return set(magnet_links)

    except FileNotFoundError:
        # Handle file not found error
        print(f"File not found: {file_name}")
        return set([])

In [ ]:
def get_magnet_links(existing_links: set) -> set:
    """
    Get unique magnet links from user input and validate them.

    Args:
        existing_links (set): Set containing already existing magnet links.

    Returns:
        set: Updated set of magnet links.
    """
    count = 0
    timeout = 60  # seconds
    start_time = time.time()

    try:
        while True:
            link = input("Enter a magnet link or 'exit' to quit: ")

            if link.lower() in ["exit", "quit", "q"]:
                break

            if validate_magnet_link(link, exact=True):
                if link not in existing_links:
                    existing_links.add(link)
                    print(f"Added magnet link: {link}")
                    count += 1
                else:
                    print(f"Magnet link already added: {link}")

                start_time = time.time()  # Reset the timer
            else:
                print(f"Invalid magnet link: {link}")

            if time.time() - start_time > timeout:
                print("Timeout: No input received in the last 60 seconds.")
                break

    except KeyboardInterrupt:
        pass

    print(f"{time.strftime('%H:%M:%S')} - Added {count} magnet links")
    return existing_links

In [ ]:
def update_txt_file(file_name: str, magnet_links: set):
    # Extract folder name and file base name
    folder_name, _ = os.path.split(file_name)

    # Create directory if it doesn't exist
    create_directory(folder_name)

    # Write magnet links to the file
    with open(file_name, "w") as f:
        f.write("\n".join(magnet_links))

    # Print a timestamped message
    print(f"{time.strftime('%H:%M:%S')} - Txt file updated -> {file_name}")

In [ ]:
class Torrent:
    def __init__(
        self,
        ses: lt.session,
        magnet_link: str,
    ):
        self.magnet_link = magnet_link
        self.save_path = "/content/Torrents"
        self.timestamp = time.time()
        self.handle = self.add_torrent(ses)
        self.name = self.handle.status().name

    def add_torrent(self, ses: lt.session):
        atp = lt.parse_magnet_uri(self.magnet_link)
        atp.save_path = self.save_path
        atp.storage_mode = lt.storage_mode_t.storage_mode_sparse

        return ses.add_torrent(atp)

    def remove_torrent(self, ses: lt.session):
        ses.remove_torrent(self.handle)
        print(f"Torrent successfully removed: {self.name}")

    def remove_stale_torrents(self, ses: lt.session):
        if time.time() - self.timestamp > 7200:
            self.remove_torrent(ses)
            return True

        return False

    def get_status(self):
        return self.handle.status()

    def get_folder_name(self, lower=True):
        # Extract strings and numbers using regex
        name = (
            self.name[:50].split(".")[0]
            if "." in self.name
            else self.name[:50].split(" ")[0]
        )

        folder_name = re.sub(r"[^a-zA-Z0-9]", "", name)

        # Optionally convert to lowercase
        folder_name = folder_name.lower() if lower else folder_name

        return folder_name

    def get_torrent_name(self):
        return self.handle.status().name

    def remove_the_link_from_the_list(self, magnet_links):
        magnet_links.remove(self.magnet_link)

    def remove_files(self):
        # Format the folder name for matching, limiting to the first 40 characters
        folder_name_match = re.sub(r"\s+", ".", self.name)[:40]

        try:
            # Find the source folder matching the formatted name
            for folder in os.listdir(self.save_path):
                if folder_name_match in folder:
                    source_folder = os.path.join(self.save_path, folder)
                    break
            else:
                # If loop completes without finding a match
                raise FileNotFoundError

            shutil.rmtree(source_folder)
            print(f"Files successfully removed from session: {source_folder}")

        except Exception as e:
            print(f"Session does not contain files. {e}")

    def move_files(self, destination):
        shutil.move(
            os.path.join(self.save_path, self.get_folder_name()),
            os.path.join(destination, self.get_folder_name()),
        )

    def move_video_files(self, destination: str):
        # Create the destination folder for the torrent
        torrent_destination = os.path.join(destination, self.get_folder_name())
        create_directory(torrent_destination)

        # Initialize a counter for moved files
        moved_files = 0

        # Format the folder name for matching, limiting to the first 40 characters
        folder_name_match = re.sub(r"\s+", ".", self.name)[:40]

        # if folder name match is empty print error message and return None
        if not folder_name_match:
            # add torrent info to the message
            print(
                f"{time.strftime('%H:%M:%S')} - Folder name match is empty! Can't move files. - Torrent name: {self.name}"
            )
            return False

        source_folder = None

        # Find the source folder matching the formatted name
        for folder in os.listdir(self.save_path):
            if folder_name_match in folder:
                source_folder = os.path.join(self.save_path, folder)
                break

        # If loop completes without finding a match print error message and return None
        if not source_folder:
            print(
                f"{time.strftime('%H:%M:%S')} - Can't find source folder at the save path! Can't move files. - Torrent name: {self.name}"
            )
            return False

        # Move video files with specific extensions
        for file in os.listdir(source_folder):
            if file.lower().endswith((".mp4", ".mkv", ".avi", ".mov")):
                source_path = os.path.join(source_folder, file)
                destination_path = os.path.join(torrent_destination, file)

                # Move the file to the destination folder
                shutil.move(source_path, destination_path)
                moved_files += 1  # Increment the counter
                time.sleep(1)  # Sleep for 1 second

        # Remove the source folder after moving files
        shutil.rmtree(source_folder)
        time.sleep(1)

        # Print a message indicating the number of moved files and the destination folder
        print(f"Moved {moved_files} video files to: {torrent_destination}")

        # Return true if at least one file is moved to remove the torrent from the list
        return moved_files > 0

In [ ]:
def create_torrents(ses: lt.session, magnet_links: set, downloads: list) -> set:
    """
    Create and add new torrents to the session.

    Args:
    - ses (lt.session): libtorrent session
    - magnet_links (set): Set of magnet links
    - downloads (list): List of existing torrent objects

    Returns:
    - set: Updated set of magnet links
    """

    print(
        f"{time.strftime('%H:%M:%S')} - Starting to create and add torrents to the session."
    )

    created_count = 0

    for magnet_link in magnet_links:
        if magnet_link in [torrent.magnet_link for torrent in downloads]:
            print(f"Torrent already added to session: {magnet_link[:120]}")

        else:
            downloads.append(Torrent(ses, magnet_link))
            created_count += 1
            # Increase sleep time for stability
            time.sleep(5)

    print(
        f"{time.strftime('%H:%M:%S')} - Successfully created {created_count} new torrents."
    )

In [ ]:
def check_already_downladed(
    ses: lt.session,
    downloads: list,
    destination: str,
    magnet_links: set,
    magnet_links_txt: str,
):
    print(f"{time.strftime('%H:%M:%S')} - Checking if a torrent already downloaded..")

    for torrent in downloads:
        folder_path = os.path.join(destination, torrent.get_folder_name())

        if os.path.exists(folder_path):
            downloadeds = os.listdir(folder_path)

            if any(torrent.name[:50] in downloaded[:50] for downloaded in downloadeds):
                torrent.remove_torrent(ses)
                torrent.remove_files()
                torrent.remove_the_link_from_the_list(magnet_links)
                update_txt_file(magnet_links_txt, magnet_links)
                torrent_name = torrent.name
                downloads.remove(torrent)
                print(
                    f"Torrent already downloaded: {torrent_name} and removed from session!!!\nIt's already in {folder_path}."
                )
                gc.collect()

        # if the folder not created create a folder
        else:
            create_directory(folder_path)
            print(f"Directory created for file: {torrent.name[:80]} to {folder_path}")

In [ ]:
def download_torrents(
    ses: lt.session,
    downloads: list,
    destination: str,
    magnet_links: set,
    magnet_links_txt: str,
):
    """
    Download torrents and perform cleanup based on conditions.

    Args:
    - ses (lt.session): libtorrent session
    - downloads (list): List of torrent objects
    - destination (str): Destination folder for downloaded files
    - magnet_links (set): Set of magnet links
    - magnet_links_txt (str): Path to the text file containing magnet links
    """

    print(
        f"\n{time.strftime('%H:%M:%S')} - Starting download with {len(downloads)} torrent(s)"
    )

    # Loop until all torrents are processed
    while downloads:
        for torrent in downloads.copy():
            # Check if torrent is stale and remove if necessary
            if torrent.remove_stale_torrents(ses):
                torrent.remove_files()
                torrent.remove_the_link_from_the_list(magnet_links)
                update_txt_file(magnet_links_txt, magnet_links)
                torrent_name = torrent.name
                downloads.remove(torrent)
                print(
                    f"{time.strftime('%H:%M:%S')} - Removed torrent due to 2 hours runtime: {torrent_name}"
                )
                continue

            # Check if torrent is still downloading
            if not torrent.get_status().is_seeding:
                continue

            # Torrent is successfully downloaded
            else:
                if torrent.move_video_files(destination):
                    torrent.remove_torrent(ses)
                    torrent.remove_the_link_from_the_list(magnet_links)
                    update_txt_file(magnet_links_txt, magnet_links)
                    torrent_name = torrent.name
                    downloads.remove(torrent)
                    print(
                        f"{time.strftime('%H:%M:%S')} - Successfully downloaded: {torrent_name}"
                    )

                else:
                    torrent.remove_torrent(ses)
                    torrent.remove_the_link_from_the_list(magnet_links)

                gc.collect()

        time.sleep(300)
        # Display the number of torrents still downloading
        print(f"{time.strftime('%H:%M:%S')} - Torrents downloading: {len(downloads)}")

In [ ]:
# give the path of the txt file
magnet_links_txt = r"/content/drive/MyDrive/Torrents/magnet_links.txt"

# give the path of the destination folder
destination = r"/content/drive/MyDrive/Torrents"

# global downlads list
downloads = []

settings = {
    "user_agent": f"python_client/{lt.__version__}",
    "listen_interfaces": "0.0.0.0:6881",
}
# session created
ses = lt.session(settings)

In [ ]:
def main():
    start_time = time.time()

    # load magnet links from txt file
    magnet_links = load_magnet_links(magnet_links_txt)

    # get magnet links from user input
    magnet_links = get_magnet_links(magnet_links)

    # update txt file
    update_txt_file(magnet_links_txt, magnet_links)

    # use global downloads list to keep track of torrents
    create_torrents(ses, magnet_links, downloads)

    # check if torrent already downloaded
    check_already_downladed(ses, downloads, destination, magnet_links, magnet_links_txt)

    # start downloading
    download_torrents(ses, downloads, destination, magnet_links, magnet_links_txt)

    # Calculate and display the total time taken
    end_time = time.time()
    elapsed_time_minutes = (end_time - start_time) // 60
    print(f"Torrents download completed in {elapsed_time_minutes} minutes.")

In [ ]:
if __name__ == "__main__":
    main()